In [1]:
### ORIGINAL ####

import pandas as pd
import numpy as np
from scipy.spatial import distance_matrix
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations

# Load data
df = pd.read_csv('locations copy.csv', header=None, names=['city', 'x', 'y'])

# Prompt user for current city
current_city = input("Please enter your current city (leave empty and press enter to use the first city in the list): ")

# If no city is entered, use the first city in the list
if current_city == '':
    current_city = df.iloc[0]['city']
# If a city is entered, validate input
elif current_city not in df['city'].values:
    raise ValueError("Invalid city. Make sure it's one of the cities in the dataset.")

# Calculate distance matrix
distances = distance_matrix(df[['x', 'y']].values, df[['x', 'y']].values)
dist_matrix = pd.DataFrame(distances, index=df.city, columns=df.city)

# Create graph
G = nx.from_numpy_array(dist_matrix.values.astype(float))

# Get index of current city
source_index = df[df['city'] == current_city].index[0]

# Find initial path
tsp_route = nx.approximation.greedy_tsp(G, source=source_index)

# 2-opt algorithm
for _ in range(50):  # Run for a fixed number of iterations
    for i, j in combinations(range(1, len(tsp_route) - 1), 2):  # For each pair of edges in the path
        if i != j:
            new_route = tsp_route[:i] + tsp_route[i:j][::-1] + tsp_route[j:]  # Try reversing the path between i and j
            if sum(dist_matrix.values[new_route[i], new_route[i - 1]] for i in range(1, len(new_route))) < sum(
                    dist_matrix.values[tsp_route[i], tsp_route[i - 1]] for i in range(1, len(tsp_route))):  # If the new path is shorter
                tsp_route = new_route  # Update the path

# Get city names
route = [(df.iloc[i]['city'], df.iloc[i]['x'], df.iloc[i]['y']) for i in tsp_route]

# Remove last city to make it one way
route_one_way = route[:-1]

# Save result
with open('route.txt', 'w') as f:
    for city, x, y in route_one_way:
        f.write(f'{city}, {x}, {y}\n')

# Create plot
plt.figure(figsize=(10, 10))
plt.scatter(df['x'], df['y'])

# Add labels
for _, row in df.iterrows():
    plt.text(row['x'], row['y'], row['city'])

prev_node = tsp_route[0]
for node in tsp_route[1:-1]:  # stop one city before the end
    plt.plot([df.iloc[prev_node]['x'], df.iloc[node]['x']], [df.iloc[prev_node]['y'], df.iloc[node]['y']], 'k-')
    prev_node = node
plt.show()

In [3]:
#

In [7]:
import os
import webbrowser
import pandas as pd
import numpy as np
import networkx as nx
import tkinter as tk
from tkinter import filedialog, messagebox
from itertools import combinations
from scipy.spatial import distance_matrix
import plotly.graph_objects as go

def load_dataset():
    global df
    filepath = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
    if not filepath:
        return
    df = pd.read_csv(filepath, names=['city', 'x', 'y'], skiprows=1)
    start_entry.config(state='normal')
    stop_entry.config(state='normal')
    run_button.config(state='normal')
    export_button.config(state='normal')

def export_dataset():
    if route:
        df_route = pd.DataFrame(route[:-1], columns=["city", "x", "y"])  # Exclude the last point
        filepath = filedialog.asksaveasfilename(defaultextension=".csv")
        if not filepath:
            return
        df_route.to_csv(filepath, index=False, header=False)



def run_tsp():
    global df
    global route
    current_city = start_entry.get()
    if current_city == '':
        current_city = df.iloc[0]['city']
    elif current_city not in df['city'].values:
        messagebox.showerror("Error", "Invalid city. Make sure it's one of the cities in the dataset.")
        return
    distances = distance_matrix(df[['x', 'y']].values, df[['x', 'y']].values)
    dist_matrix = pd.DataFrame(distances, index=df.city, columns=df.city)
    G = nx.from_numpy_array(dist_matrix.values.astype(float))
    source_index = df[df['city'] == current_city].index[0]
    tsp_route = nx.approximation.greedy_tsp(G, source=source_index)
    for _ in range(50):
        for i, j in combinations(range(1, len(tsp_route) - 1), 2):
            if i != j:
                new_route = tsp_route[:i] + tsp_route[i:j][::-1] + tsp_route[j:]
                if sum(dist_matrix.values[new_route[i], new_route[i - 1]] for i in range(1, len(new_route))) < sum(
                        dist_matrix.values[tsp_route[i], tsp_route[i - 1]] for i in range(1, len(tsp_route))):
                    tsp_route = new_route
    route = [(df.iloc[i]['city'], df.iloc[i]['x'], df.iloc[i]['y']) for i in tsp_route]
    route_one_way = route[:-1]
    total_distance = 0
    for i in range(len(route_one_way) - 1):
        total_distance += np.sqrt((route_one_way[i][1] - route_one_way[i + 1][1]) ** 2 + (route_one_way[i][2] - route_one_way[i + 1][2]) ** 2)
    route_info = "<br>".join([f"{r[0]}, {r[1]}, {r[2]}" for r in route_one_way])
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df['x'],
        y=df['y'],
        mode='markers+text',
        text=df['city'],
        textposition='top center'
    ))
    fig.add_trace(go.Scatter(
        x=[r[1] for r in route_one_way],
        y=[r[2] for r in route_one_way],
        mode='lines',
        line=dict(color='royalblue', width=2),
    ))
    fig.add_trace(go.Scatter(
        x=[route_one_way[0][1], route_one_way[-1][1]],
        y=[route_one_way[0][2], route_one_way[-1][2]],
        mode='markers',
        marker=dict(size=12, color=['green', 'red']),
    ))
    fig.add_annotation(
        xref="paper",
        yref="paper",
        x=1.35,
        y=0.5,
        align='left',
        text=f"Total Distance: {total_distance:.2f} m<br>{route_info}",
        showarrow=False,
        bordercolor="black",
        bgcolor="white",
        borderwidth=2,
        borderpad=4,
        font=dict(
            family="Courier New, monospace",
            size=14,
            color="#000000"
        ),
    )
    fig.update_layout(
        title="Traveling Salesperson Path",
        xaxis_title="X",
        yaxis_title="Y",
        autosize=False,
        width=1500,
        height=800,
        margin=dict(
            l=50,
            r=500,
            b=100,
            t=100,
            pad=4
        ),
        showlegend=False,
    )
    fig.write_html("plot.html")
    webbrowser.open('file://' + os.path.realpath("plot.html"))

root = tk.Tk()
root.title("TSP Solver")
df = None
route = None
load_button = tk.Button(root, text="Load Dataset", command=load_dataset)
load_button.pack()
start_entry = tk.Entry(root, state='disabled')
start_entry.pack()
stop_entry = tk.Entry(root, state='disabled')
stop_entry.pack()
run_button = tk.Button(root, text="Run TSP", state='disabled', command=run_tsp)
run_button.pack()
export_button = tk.Button(root, text="Export Route", state='disabled', command=export_dataset)
export_button.pack()
root.mainloop()


In [1]:
################ 2 OPT new mod ##############

In [25]:
import os
import webbrowser
import pandas as pd
import numpy as np
import networkx as nx
import tkinter as tk
from tkinter import filedialog, messagebox
from itertools import combinations
from scipy.spatial import distance_matrix
import plotly.graph_objects as go
from tkinter import ttk

def load_dataset():
    global df
    filepath = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
    if not filepath:
        return
    df = pd.read_csv(filepath, header=None, names=['city', 'x', 'y'])
    df['order'] = np.nan  # New column to store the order of addition
    for point in df['city']:
        points_listbox.insert('end', point)
    start_entry.config(state='normal')
    run_button.config(state='normal')
    export_button.config(state='normal')
    progress['maximum'] = 50  # The number of iterations in your TSP calculation

def add_point():
    point = points_listbox.get(points_listbox.curselection())
    points_listbox.delete(points_listbox.curselection())
    selected_points_listbox.insert('end', point)
    df.loc[df['city'] == point, 'order'] = selected_points_listbox.size()  # Record the order of addition

def remove_point():
    point = selected_points_listbox.get(selected_points_listbox.curselection())
    selected_points_listbox.delete(selected_points_listbox.curselection())
    points_listbox.insert('end', point)
    df.loc[df['city'] == point, 'order'] = np.nan  # Reset the order of addition

def export_dataset():
    if route:
        df_route = pd.DataFrame(route[:-1], columns=["city", "x", "y"])  # Exclude the last point
        filepath = filedialog.asksaveasfilename(defaultextension=".csv")
        if not filepath:
            return
        df_route.to_csv(filepath, index=False, header=False)

def run_tsp():
    global df
    global route
    selected_points = list(selected_points_listbox.get(0, 'end'))
    df_selected = df[df['city'].isin(selected_points)].sort_values('order').reset_index(drop=True)  # Sort by the order of addition
    current_city = start_entry.get()
    if current_city == '':
        current_city = df_selected.iloc[0]['city']
    elif current_city not in df_selected['city'].values:
        messagebox.showerror("Error", "Invalid Point. Make sure it's one of the Points in the dataset.")
        return
    distances = distance_matrix(df_selected[['x', 'y']].values, df_selected[['x', 'y']].values)
    dist_matrix = pd.DataFrame(distances, index=df_selected.city, columns=df_selected.city)
    G = nx.from_numpy_array(dist_matrix.values.astype(float))
    source_index = df_selected[df_selected['city'] == current_city].index[0]
    tsp_route = nx.approximation.greedy_tsp(G, source=source_index)
    progress['value'] = 0
    for _ in range(50):
        progress['value'] += 1  # Update the progress bar
        root.update_idletasks()
        for i, j in combinations(range(1, len(tsp_route) - 1), 2):
            if i != j:
                new_route = tsp_route[:i] + tsp_route[i:j][::-1] + tsp_route[j:]
                if sum(dist_matrix.values[new_route[i], new_route[i - 1]] for i in range(1, len(new_route))) < sum(
                        dist_matrix.values[tsp_route[i], tsp_route[i - 1]] for i in range(1, len(tsp_route))):
                    tsp_route = new_route
    route = [(df_selected.iloc[i]['city'], df_selected.iloc[i]['x'], df_selected.iloc[i]['y']) for i in tsp_route]
    route_one_way = route[:-1]
    total_distance = 0
    for i in range(len(route_one_way) - 1):
        total_distance += np.sqrt((route_one_way[i][1] - route_one_way[i + 1][1]) ** 2 + (route_one_way[i][2] - route_one_way[i + 1][2]) ** 2)
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df['x'],
        y=df['y'],
        mode='markers+text',
        text=df['city'],
        textposition='top center'
    ))
    fig.add_trace(go.Scatter(
        x=[r[1] for r in route_one_way],
        y=[r[2] for r in route_one_way],
        mode='lines',
        line=dict(color='royalblue', width=2),
    ))
    fig.add_trace(go.Scatter(
        x=[route_one_way[0][1], route_one_way[-1][1]],
        y=[route_one_way[0][2], route_one_way[-1][2]],
        mode='markers',
        marker=dict(size=12, color=['green', 'red']),
    ))
    fig.update_layout(
        title=f"2opt method - Total Distance: {total_distance:.2f}",
        xaxis_title="X",
        yaxis_title="Y",
        autosize=False,
        width=1500,
        height=800,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),
        showlegend=False,
    )
    fig.write_html("plot.html")
    webbrowser.open('file://' + os.path.realpath("plot.html"))
    progress['value'] = 0

root = tk.Tk()
root.title("TSP Solver")
df = None
route = None
load_button = tk.Button(root, text="Load Dataset", command=load_dataset)
load_button.pack()
points_listbox = tk.Listbox(root)
points_listbox.pack()
add_button = tk.Button(root, text="Add", command=add_point)
add_button.pack()
selected_points_listbox = tk.Listbox(root)
selected_points_listbox.pack()
remove_button = tk.Button(root, text="Remove", command=remove_point)
remove_button.pack()
start_entry = tk.Entry(root, state='disabled')
start_entry.pack()
run_button = tk.Button(root, text="Run TSP", state='disabled', command=run_tsp)
run_button.pack()
export_button = tk.Button(root, text="Export Route", state='disabled', command=export_dataset)
export_button.pack()
progress = ttk.Progressbar(root, orient='horizontal', length=300, mode='determinate')
progress.pack()
root.mainloop()



In [18]:
########### ABOVE IS OK

In [19]:
############ TRYING TO INVOLVE ALL POINTS#

In [26]:
import os
import webbrowser
import pandas as pd
import numpy as np
import networkx as nx
import tkinter as tk
from tkinter import filedialog, messagebox
from itertools import combinations
from scipy.spatial import distance_matrix
import plotly.graph_objects as go
from tkinter import ttk
from ttkwidgets import CheckboxTreeview

def load_dataset():
    global df
    filepath = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
    if not filepath:
        return
    df = pd.read_csv(filepath, header=None, names=['city', 'x', 'y'])
    df['order'] = np.nan  # New column to store the order of addition
    for point in df['city']:
        points_treeview.insert("", "end", point, text=point)
    start_entry.config(state='normal')
    run_button.config(state='normal')
    export_button.config(state='normal')
    progress['maximum'] = 50  # The number of iterations in your TSP calculation

def add_point():
    points = points_treeview.get_checked()
    for point in points:
        points_treeview.delete(point)
        selected_points_treeview.insert("", "end", point, text=point)
        df.loc[df['city'] == point, 'order'] = selected_points_treeview.get_children().__len__()  # Record the order of addition

def remove_point():
    points = selected_points_treeview.get_checked()
    for point in points:
        selected_points_treeview.delete(point)
        points_treeview.insert("", "end", point, text=point)
        df.loc[df['city'] == point, 'order'] = np.nan  # Reset the order of addition

def export_dataset():
    if route:
        df_route = pd.DataFrame(route[:-1], columns=["city", "x", "y"])  # Exclude the last point
        filepath = filedialog.asksaveasfilename(defaultextension=".csv")
        if not filepath:
            return
        df_route.to_csv(filepath, index=False, header=False)

def run_tsp():
    global df
    global route
    selected_points = list(selected_points_treeview.get_children())
    df_selected = df[df['city'].isin(selected_points)].sort_values('order').reset_index(drop=True)  # Sort by the order of addition
    current_city = start_entry.get()
    if current_city == '':
        current_city = df_selected.iloc[0]['city']
    elif current_city not in df_selected['city'].values:
        messagebox.showerror("Error", "Invalid Point. Make sure it's one of the Points in the dataset.")
        return
    distances = distance_matrix(df_selected[['x', 'y']].values, df_selected[['x', 'y']].values)
    dist_matrix = pd.DataFrame(distances, index=df_selected.city, columns=df_selected.city)
    G = nx.from_numpy_array(dist_matrix.values.astype(float))
    source_index = df_selected[df_selected['city'] == current_city].index[0]
    tsp_route = nx.approximation.greedy_tsp(G, source=source_index)
    progress['value'] = 0
    for _ in range(50):
        progress['value'] += 1  # Update the progress bar
        root.update_idletasks()
        for i, j in combinations(range(1, len(tsp_route) - 1), 2):
            if i != j:
                new_route = tsp_route[:i] + tsp_route[i:j][::-1] + tsp_route[j:]
                if sum(dist_matrix.values[new_route[i], new_route[i - 1]] for i in range(1, len(new_route))) < sum(
                        dist_matrix.values[tsp_route[i], tsp_route[i - 1]] for i in range(1, len(tsp_route))):
                    tsp_route = new_route
    route = [(df_selected.iloc[i]['city'], df_selected.iloc[i]['x'], df_selected.iloc[i]['y']) for i in tsp_route]
    route_one_way = route[:-1]
    total_distance = 0
    for i in range(len(route_one_way) - 1):
        total_distance += np.sqrt((route_one_way[i][1] - route_one_way[i + 1][1]) ** 2 + (route_one_way[i][2] - route_one_way[i + 1][2]) ** 2)
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df['x'],
        y=df['y'],
        mode='markers+text',
        text=df['city'],
        textposition='top center'
    ))
    fig.add_trace(go.Scatter(
        x=[r[1] for r in route_one_way],
        y=[r[2] for r in route_one_way],
        mode='lines',
        line=dict(color='royalblue', width=2),
    ))
    fig.add_trace(go.Scatter(
        x=[route_one_way[0][1], route_one_way[-1][1]],
        y=[route_one_way[0][2], route_one_way[-1][2]],
        mode='markers',
        marker=dict(size=12, color=['green', 'red']),
    ))
    fig.update_layout(
        title=f"2opt method - Total Distance: {total_distance:.2f}",
        xaxis_title="X",
        yaxis_title="Y",
        autosize=False,
        width=1500,
        height=800,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),
        showlegend=False,
    )
    fig.write_html("plot.html")
    webbrowser.open('file://' + os.path.realpath("plot.html"))
    progress['value'] = 0

root = tk.Tk()
root.title("TSP Solver")
df = None
route = None
load_button = tk.Button(root, text="Load Dataset", command=load_dataset)
load_button.pack()
points_treeview = CheckboxTreeview(root)
points_treeview.pack()
add_button = tk.Button(root, text="Add", command=add_point)
add_button.pack()
selected_points_treeview = CheckboxTreeview(root)
selected_points_treeview.pack()
remove_button = tk.Button(root, text="Remove", command=remove_point)
remove_button.pack()
start_entry = tk.Entry(root, state='disabled')
start_entry.pack()
run_button = tk.Button(root, text="Run TSP", state='disabled', command=run_tsp)
run_button.pack()
export_button = tk.Button(root, text="Export Route", state='disabled', command=export_dataset)
export_button.pack()
progress = ttk.Progressbar(root, orient='horizontal', length=300, mode='determinate')
progress.pack()
root.mainloop()


In [ ]:
############## WORKS FINE

In [27]:
import os
import webbrowser
import pandas as pd
import numpy as np
import networkx as nx
import tkinter as tk
from tkinter import filedialog, messagebox
from itertools import combinations
from scipy.spatial import distance_matrix
import plotly.graph_objects as go
from tkinter import ttk
from ttkwidgets import CheckboxTreeview

def load_dataset():
    global df
    filepath = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
    if not filepath:
        return
    df = pd.read_csv(filepath, header=None, names=['city', 'x', 'y'])
    df['order'] = np.nan  # New column to store the order of addition
    for point in df['city']:
        points_treeview.insert("", "end", point, text=point)
    start_entry.config(state='normal')
    run_button.config(state='normal')
    export_button.config(state='normal')
    progress['maximum'] = 50  # The number of iterations in your TSP calculation

def add_point():
    points = points_treeview.get_checked()
    for point in points:
        points_treeview.delete(point)
        selected_points_treeview.insert("", "end", point, text=point)
        df.loc[df['city'] == point, 'order'] = selected_points_treeview.get_children().__len__()  # Record the order of addition

def remove_point():
    points = selected_points_treeview.get_checked()
    for point in points:
        selected_points_treeview.delete(point)
        points_treeview.insert("", "end", point, text=point)
        df.loc[df['city'] == point, 'order'] = np.nan  # Reset the order of addition

def export_dataset():
    if route:
        df_route = pd.DataFrame(route[:-1], columns=["city", "x", "y"])  # Exclude the last point
        filepath = filedialog.asksaveasfilename(defaultextension=".csv")
        if not filepath:
            return
        df_route.to_csv(filepath, index=False, header=False)

def run_tsp():
    global df
    global route
    selected_points = list(selected_points_treeview.get_children())
    df_selected = df[df['city'].isin(selected_points)].sort_values('order').reset_index(drop=True)  # Sort by the order of addition
    current_city = start_entry.get()
    if current_city == '':
        current_city = df_selected.iloc[0]['city']
    elif current_city not in df_selected['city'].values:
        messagebox.showerror("Error", "Invalid Point. Make sure it's one of the Points in the dataset.")
        return
    distances = distance_matrix(df_selected[['x', 'y']].values, df_selected[['x', 'y']].values)
    dist_matrix = pd.DataFrame(distances, index=df_selected.city, columns=df_selected.city)
    G = nx.from_numpy_array(dist_matrix.values.astype(float))
    source_index = df_selected[df_selected['city'] == current_city].index[0]
    tsp_route = nx.approximation.greedy_tsp(G, source=source_index)
    progress['value'] = 0
    for _ in range(50):
        progress['value'] += 1  # Update the progress bar
        root.update_idletasks()
        for i, j in combinations(range(1, len(tsp_route) - 1), 2):
            if i != j:
                new_route = tsp_route[:i] + tsp_route[i:j][::-1] + tsp_route[j:]
                if sum(dist_matrix.values[new_route[i], new_route[i - 1]] for i in range(1, len(new_route))) < sum(
                        dist_matrix.values[tsp_route[i], tsp_route[i - 1]] for i in range(1, len(tsp_route))):
                    tsp_route = new_route
    route = [(df_selected.iloc[i]['city'], df_selected.iloc[i]['x'], df_selected.iloc[i]['y']) for i in tsp_route]
    route_one_way = route[:-1]
    total_distance = 0
    for i in range(len(route_one_way) - 1):
        total_distance += np.sqrt((route_one_way[i][1] - route_one_way[i + 1][1]) ** 2 + (route_one_way[i][2] - route_one_way[i + 1][2]) ** 2)
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df['x'],
        y=df['y'],
        mode='markers+text',
        text=df['city'],
        textposition='top center'
    ))
    fig.add_trace(go.Scatter(
        x=[r[1] for r in route_one_way],
        y=[r[2] for r in route_one_way],
        mode='lines',
        line=dict(color='royalblue', width=2),
    ))
    fig.add_trace(go.Scatter(
        x=[route_one_way[0][1], route_one_way[-1][1]],
        y=[route_one_way[0][2], route_one_way[-1][2]],
        mode='markers',
        marker=dict(size=12, color=['green', 'red']),
    ))
    fig.update_layout(
        title=f"2opt method - Total Distance: {total_distance:.2f}",
        xaxis_title="X",
        yaxis_title="Y",
        autosize=False,
        width=1500,
        height=800,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),
        showlegend=False,
    )
    fig.write_html("plot.html")
    webbrowser.open('file://' + os.path.realpath("plot.html"))
    progress['value'] = 0

root = tk.Tk()
root.title("TSP Solver")
df = None
route = None
load_button = tk.Button(root, text="Load Dataset", command=load_dataset)
load_button.pack()
points_treeview = CheckboxTreeview(root)
points_treeview.pack()
buttons_frame = tk.Frame(root)
buttons_frame.pack()
add_button = tk.Button(buttons_frame, text="Add", command=add_point)
add_button.pack(side="left")
remove_button = tk.Button(buttons_frame, text="Remove", command=remove_point)
remove_button.pack(side="left")
selected_points_treeview = CheckboxTreeview(root)
selected_points_treeview.pack()
start_entry = tk.Entry(root, state='disabled')
start_entry.pack()
start_label = tk.Label(root, text="Optional start point: If not set, the first in the list will be chosen. You can set a custom start point if desired.")
start_label.pack()
run_button = tk.Button(root, text="Run TSP", state='disabled', command=run_tsp)
run_button.pack()
export_button = tk.Button(root, text="Export Route", state='disabled', command=export_dataset)
export_button.pack()
progress = ttk.Progressbar(root, orient='horizontal', length=300, mode='determinate')
progress.pack()
root.mainloop()


In [ ]:
#BUENO